In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [2]:
with open("pose_data.json") as f:
    data = json.load(f)

In [3]:
def get_velocity(data, vid_index):
    '''
    calculate the velocity by subtracting frame by previous frame
    :param data:
    :param vid_index:
    :return:
    '''
    velocities = []
    for frame_num in range(1, len(data[vid_index])):
        this_frame = np.array(data[vid_index][frame_num]['keypoints'])[:, :2]      # keep x,y
        prev_frame = np.array(data[vid_index][frame_num - 1]['keypoints'])[:, :2]  # keep x,y
        velocity = this_frame - prev_frame
        velocities.append(velocity)
    return np.array(velocities)
get_velocity(data, 0)[0]

array([[ -5.68579102,  -0.30340576],
       [ -6.68170166,   0.42456055],
       [ -7.22808838,   0.78613281],
       [ -5.85461426,   2.22869873],
       [ -6.16943359,   4.95825195],
       [ -3.72705078,   3.57012939],
       [ -0.57525635,   3.02185059],
       [ -6.99530029,   1.45544434],
       [ -0.43304443,  -2.51104736],
       [ -3.48321533,  -0.29498291],
       [  6.04693604,  -8.69439697],
       [ -9.11187744,   2.90911865],
       [ -7.18566895,   1.08966064],
       [ -5.44219971,  14.96594238],
       [ -1.70635986,   7.34680176],
       [ -4.89483643,   0.        ],
       [-11.56439209,   0.        ]])

In [4]:
def get_positions(data, vid_index):
    positions = []
    for frame in data[vid_index][:-1]:
        keypoints = np.array(frame['keypoints'])  # shape: (num_keypoints, 3)
        xy = keypoints[:, :2]                     # keep only x,y
        positions.append(xy)
    return np.array(positions)

get_positions(data, 0)[0]

array([[ 862.15374756,  535.06195068],
       [ 877.99755859,  517.71368408],
       [ 840.0526123 ,  515.70220947],
       [ 891.92041016,  538.27392578],
       [ 795.77337646,  533.52441406],
       [ 929.55651855,  662.74713135],
       [ 740.66021729,  664.88897705],
       [ 969.35119629,  812.37463379],
       [ 706.76324463,  827.85131836],
       [ 995.80078125,  943.01055908],
       [ 715.30529785,  963.19146729],
       [ 913.52166748,  979.58111572],
       [ 790.7142334 ,  984.61785889],
       [ 911.94378662, 1182.68798828],
       [ 790.85412598, 1192.65319824],
       [ 873.44903564, 1200.        ],
       [ 800.50354004, 1200.        ]])

In [5]:
data[0][1]['keypoints'][0][:2]

[856.4679565429688, 534.758544921875]

In [6]:
def normalize(data):
    vel = data.astype(float).copy()
    max_xy = np.abs(vel).max(axis=tuple(range(vel.ndim - 1)), keepdims=True)
    max_xy = np.maximum(max_xy, 1e-8)
    return vel / max_xy

In [7]:
def get_pos_vel(data, vid_index):
    pos = get_positions(data, vid_index)  # shape (..., Dp)
    vel = get_velocity(data, vid_index)   # shape (..., Dv)
    return np.concatenate([pos, vel], axis=-1)

In [8]:
pos_val = get_pos_vel(data, 0)[0]
normalize(pos_val)

array([[ 0.86578939,  0.44588496, -0.49166363, -0.02027308],
       [ 0.88170001,  0.43142807, -0.57778235,  0.02836845],
       [ 0.84359505,  0.42975184, -0.62502969,  0.05252812],
       [ 0.89568157,  0.4485616 , -0.50626217,  0.14891803],
       [ 0.79912909,  0.44460368, -0.53348534,  0.33130235],
       [ 0.93347639,  0.55228928, -0.32228679,  0.23855026],
       [ 0.74378353,  0.55407415, -0.04974376,  0.20191516],
       [ 0.97343888,  0.67697886, -0.60489996,  0.09725043],
       [ 0.70974361,  0.6898761 , -0.03744636, -0.16778411],
       [ 1.        ,  0.78584213, -0.30120177, -0.01971028],
       [ 0.71832169,  0.80265956,  0.52289269, -0.58094551],
       [ 0.91737392,  0.8163176 , -0.78792533,  0.19438259],
       [ 0.79404862,  0.82051488, -0.62136158,  0.07280936],
       [ 0.91578939,  0.98557332, -0.47059972,  1.        ],
       [ 0.7941891 ,  0.99387767, -0.14755292,  0.49090138],
       [ 0.87713231,  1.        , -0.42326794,  0.        ],
       [ 0.8038792 ,  1.

# Training Stuff

In [9]:
# gesture, vid, frame
for kp in data:
    frames = len(kp)
    gesture = kp[0]['gesture_id']
    person = kp[0]['person_id']
    path = kp[0]['vid_path']
    print(f"gesture: {gesture}, \tframes: {frames},  \tpath: {path}")

print(len(data))

gesture: 0, 	frames: 29,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_4.mp4
gesture: 0, 	frames: 22,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_5.mp4
gesture: 0, 	frames: 23,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_6.mp4
gesture: 0, 	frames: 41,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_2.mp4
gesture: 0, 	frames: 42,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_3.mp4
gesture: 0, 	frames: 43,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_1.mp4
gesture: 1, 	frames: 48,  	path: Images/small/hand_fand/hand_fand_Subject1_Tuan_cam-1-index_4(1).mp4
gesture: 1, 	frames: 26,  	path: Images/small/hand_fand/hand_fand_Subject1_Tuan_cam-1-index_5.mp4
gesture: 1, 	frames: 24,  	path: Images/small/hand_fand/hand_fand_Subject1_Tuan_cam-1-index_4.mp4
gesture: 1, 	frames: 1,  	path: Images/small/hand_fand/hand_fand_Subject1_Tuan_cam-1-index_1.mp4
gesture: 1, 	frames: 47,  	path: Images/small/hand_fand/hand_fand_Subject1_Tuan_cam-1-inde

In [38]:
# https://www.geeksforgeeks.org/python/filter-in-python/

def more_than_one_frame(kp):
    return len(kp) > 1

filtered = list(filter(more_than_one_frame, data))

max_vid_length = max(len(kp) for kp in filtered)
print(max_vid_length)

for kp in filtered:
    frames = len(kp)
    gesture = kp[0]['gesture_id']
    person = kp[0]['person_id']
    path = kp[0]['vid_path']
    print(f"gesture: {gesture}, \tframes: {frames},  \tpath: {path}")

print("\nexample on one frame")
print(kp[0]['keypoints'])

68
gesture: 0, 	frames: 29,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_4.mp4
gesture: 0, 	frames: 22,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_5.mp4
gesture: 0, 	frames: 23,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_6.mp4
gesture: 0, 	frames: 41,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_2.mp4
gesture: 0, 	frames: 42,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_3.mp4
gesture: 0, 	frames: 43,  	path: Images/small/VAR/VAR_Subject1_Tuan_cam-1-index_1.mp4
gesture: 1, 	frames: 48,  	path: Images/small/hand_fand/hand_fand_Subject1_Tuan_cam-1-index_4(1).mp4
gesture: 1, 	frames: 26,  	path: Images/small/hand_fand/hand_fand_Subject1_Tuan_cam-1-index_5.mp4
gesture: 1, 	frames: 24,  	path: Images/small/hand_fand/hand_fand_Subject1_Tuan_cam-1-index_4.mp4
gesture: 1, 	frames: 47,  	path: Images/small/hand_fand/hand_fand_Subject1_Tuan_cam-1-index_3.mp4
gesture: 1, 	frames: 39,  	path: Images/small/hand_fand/hand_fand_Subject1_Tuan_cam-1-